## Home assignment 05: Bagging and OOB score

Please, fill the lines in the code below.
This is a simplified version of `BaggingRegressor` from `sklearn`. Please, notice, that `sklearn` API is **not preserved**.

Your algorithm should be able to train different instances of the same model class on bootstrapped datasets and to provide [OOB score](https://en.wikipedia.org/wiki/Out-of-bag_error) for the training set.

The model should be passed as model class with no explicit parameters and no parentheses.

Example:
```
import numpy as np
from sklearn.linear_model import LinearRegression

bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
bagging_regressor.fit(LinearRegression, X, y)

```

In [1]:
import numpy as np

In [2]:
class SimplifiedBaggingRegressor:
    def __init__(self, num_bags, oob=False):
        self.num_bags = num_bags  # Количество бэгов (подвыборок)
        self.oob = oob  # Флаг использования OOB-оценки
        
    def _generate_splits(self, data: np.ndarray):
        '''
        Генерация индексов для каждого бэга и сохранение их в self.indices_list
        '''
        self.indices_list = []
        data_length = len(data)
        for _ in range(self.num_bags):
            indices = np.random.choice(data_length, size=data_length, replace=True)  # Бустрэп-выборка
            self.indices_list.append(indices)
        
    def fit(self, model_constructor, data, target):
        '''
        Обучение модели на каждом бэге.
        
        model_constructor — класс модели, передается без параметров и без скобок.
        Пример:
        bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
        bagging_regressor.fit(LinearRegression, X, y)
        '''
        self.data = None
        self.target = None
        self._generate_splits(data)
        assert len(set(list(map(len, self.indices_list)))) == 1, 'Все бэги должны быть одинаковой длины!'
        assert list(map(len, self.indices_list))[0] == len(data), 'Все бэги должны содержать len(data) элементов!'
        
        self.models_list = []
        for indices in self.indices_list:
            model = model_constructor()
            data_bag, target_bag = data[indices], target[indices]  # Создание обучающего набора для данного бэга
            self.models_list.append(model.fit(data_bag, target_bag))  # Обучение и сохранение модели
        
        if self.oob:
            self.data = data
            self.target = target
        
    def predict(self, data):
        '''
        Получение среднего предсказания для каждого объекта из переданного набора данных
        '''
        predictions = np.array([model.predict(data) for model in self.models_list])
        return np.mean(predictions, axis=0)
    
    def _get_oob_predictions_from_every_model(self):
        '''
        Генерация списка списков, где список i содержит предсказания для объекта self.data[i]
        от всех моделей, которые не видели этот объект во время обучения
        '''
        list_of_predictions_lists = [[] for _ in range(len(self.data))]
        
        for model, indices in zip(self.models_list, self.indices_list):
            oob_mask = np.ones(len(self.data), dtype=bool)
            oob_mask[indices] = False  # Отмечаем индексы, использованные в обучении
            oob_predictions = model.predict(self.data[oob_mask])  # Предсказания для OOB-набора
            
            for i, pred in zip(np.where(oob_mask)[0], oob_predictions):
                list_of_predictions_lists[i].append(pred)
        
        self.list_of_predictions_lists = np.array(list_of_predictions_lists, dtype=object)
    
    def _get_averaged_oob_predictions(self):
        '''
        Вычисление среднего предсказания для каждого объекта обучающего набора.
        Если объект использовался во всех бэгах, возвращается None вместо предсказания.
        '''
        self._get_oob_predictions_from_every_model()
        
        self.oob_predictions = np.array([
            np.mean(preds) if len(preds) > 0 else None for preds in self.list_of_predictions_lists
        ])
    
    def OOB_score(self):
        '''
        Вычисление среднеквадратичной ошибки (MSE) для всех объектов,
        которые имеют хотя бы одно OOB-предсказание.
        '''
        self._get_averaged_oob_predictions()
        mask = self.oob_predictions != None  # Выбираем только те объекты, у которых есть OOB-предсказания
        errors = [(self.target[i] - self.oob_predictions[i]) ** 2 for i in range(len(self.target)) if mask[i]]
        return sum(errors) / len(errors) if errors else None


### Local tests:

In [3]:
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm

#### Simple tests:

In [4]:
for _ in tqdm(range(100)):
    X = np.random.randn(2000, 10)
    y = np.mean(X, axis=1)
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    assert np.mean((predictions - y)**2) < 1e-6, 'Linear dependency should be fitted with almost zero error!'
    assert bagging_regressor.oob, 'OOB feature must be turned on'
    oob_score = bagging_regressor.OOB_score()
    assert oob_score < 1e-6, 'OOB error for linear dependency should be also close to zero!'
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 0.1, 'Probability of missing a bag should be close to theoretical value!'
    
print('Simple tests done!')

  0%|          | 0/100 [00:00<?, ?it/s]

Simple tests done!


#### Medium tests

In [5]:
for _ in tqdm(range(10)):
    X = np.random.randn(200, 150)
    y = np.random.randn(len(X))
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=20, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    average_train_error = np.mean((predictions - y)**2)
    assert bagging_regressor.oob, 'OOB feature must be turned on'
    oob_score = bagging_regressor.OOB_score()
    assert oob_score > average_train_error, 'OOB error must be higher than train error due to overfitting!'
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 0.1, 'Probability of missing a bag should be close to theoretical value!'
    
print('Medium tests done!')

  0%|          | 0/10 [00:00<?, ?it/s]

Medium tests done!


#### Complex tests:

In [6]:
for _ in tqdm(range(10)):
    X = np.random.randn(2000, 15)
    y = np.random.randn(len(X))
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=100, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    oob_score = bagging_regressor.OOB_score()
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 1e-2, 'Probability of missing a bag should be close to theoretical value!'
    
print('Complex tests done!')

  0%|          | 0/10 [00:00<?, ?it/s]

Complex tests done!


In [7]:
np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)

9.555882855766251e-05

Great job! Please, save `SimplifiedBaggingRegressor` to  `bagging.py` and submit your solution to the grading system!